In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 16.3 MB/s eta 0:00:00


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kangaonkaggle/videos")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/videos


In [3]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO, solutions
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Sample Video Path
video_path = '/kaggle/input/videos/highway.mp4'

In [5]:
# Capture Video
cap = cv2.VideoCapture(video_path)

In [6]:
# Get Video's Properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

properties = {
    "Width": width, 
    "Height": height, 
    "FPS": fps}

for prop in properties:
    print(f"This is {video_path.split('/')[-1]} {prop}: {properties[prop]}")

This is highway.mp4 Width: 1920
This is highway.mp4 Height: 1080
This is highway.mp4 FPS: 29


In [7]:
# region_points = [(20, 400), (1080, 400)] for line counting
region_points = [
    (430, 700),
    (1600, 700),
    (1600, 1080),
    (430, 1080)
] # For rectangle region counting: 
    # top left, top right, bottom right, bottom left

In [8]:
# Video Writer
video_writer = cv2.VideoWriter(
    "./run/highway_counted.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, 
    (width, height)
)

In [9]:
# Call ObjectCounter
counter = solutions.ObjectCounter(
    show = False, # Display output = false
    region = region_points, # Pass region points
    model="yolo11l.pt" # Use YOLO11 OBB if possible | model = 'yolo11n-obb.pt'
)

Ultralytics Solutions: ✅ {'region': [(430, 700), (1600, 700), (1600, 1080), (430, 1080)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'show': False, 'model': 'yolo11l.pt'}


100%|██████████| 49.0M/49.0M [00:00<00:00, 222MB/s]


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



In [10]:
# Process video frames
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Video frame is empty or video processing has been completed")
        break
    frame = counter.count(frame)
    video_writer.write(frame)

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.5s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 22 cars, 2 buss, 6 trucks, 4 traffic lights, 97.6ms
Speed: 10.0ms preprocess, 97.6ms inference, 256.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 2 buss, 6 trucks, 4 traffic lights, 31.1ms
Speed: 3.1ms preprocess, 31.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 2 buss, 5 trucks, 4 traffic lights, 31.1ms
Speed: 3.0ms preprocess, 31.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 2 buss, 5 trucks, 4 traffic lights, 31.1ms
Speed: 2.5ms preprocess, 31.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 2 buss, 

In [11]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()